In [1]:
import numpy as np
import os
import re
from tqdm import tqdm_notebook
import pickle

In [2]:
DATA_DIR = r'D:\projects\nlp\summarization\stories'
VOCAB_DIR = r'D:\projects\nlp\summarization\stories\vocab'
SAVE_PKL_DIR = r'D:\projects\nlp\summarization\stories\all_pkl'
SAVE_BIN_DIR = r'D:\projects\nlp\summarization\stories\all_bin'
RESTART = True
BATCH_SIZE = 16

In [3]:
re_perc_1 = re.compile('[0-9]+[.]?[0-9]+\s*%')
re_curr_1 = re.compile('[0-9]+[.]?[0-9]+\s*[лв|CHF|Kč|kr|€|£|kn|₾|ft|kr|zł|₽|lei|kr|₺|₴|د.إ|₪|Ksh|.د.م|₦|R|R$|$|$|$|$|S/.|$|$|৳|¥|元|HK$|Rp|₹|¥|RM|$|₱|Rs|$|₩|Rs|฿|₫|₿|₿|XRP|ɱ|Ł|Ξ|€|£|$|¥] ')
re_curr_2 = re.compile(' [лв|CHF|Kč|kr|€|£|kn|₾|ft|kr|zł|₽|lei|kr|₺|₴|د.إ|₪|Ksh|.د.م|₦|R|R$|$|$|$|$|S/.|$|$|৳|¥|元|HK$|Rp|₹|¥|RM|$|₱|Rs|$|₩|Rs|฿|₫|₿|₿|XRP|ɱ|Ł|Ξ|€|£|$|¥]\s*[0-9]+[.]?[0-9]+')
re_curr_3 = re.compile('[0-9]+[.]?[0-9]+\s*[BGN|CHF|CZK|DKK|EUR|GBP|HRK|GEL|HUF|NOK|PLN|RUB|RON|SEK|TRY|UAH|AED|ILS|KES|MAD|NGN|ZAR|BRL|CAD|CLP|COP|MXN|PEN|USD|AUD|BDT|CNY|CNY|HKD|IDR|INR|JPY|MYR|NZD|PHP|PKR|SGD|KRW|LKR|THB|VND|XBT|BTC|XRP|XMR|LTC|ETH|EUR|GBP|USD|JPY] ')
re_curr_4 = re.compile(' [BGN|CHF|CZK|DKK|EUR|GBP|HRK|GEL|HUF|NOK|PLN|RUB|RON|SEK|TRY|UAH|AED|ILS|KES|MAD|NGN|ZAR|BRL|CAD|CLP|COP|MXN|PEN|USD|AUD|BDT|CNY|CNY|HKD|IDR|INR|JPY|MYR|NZD|PHP|PKR|SGD|KRW|LKR|THB|VND|XBT|BTC|XRP|XMR|LTC|ETH|EUR|GBP|USD|JPY]\s*[0-9]+[.]?[0-9]+')
re_numb_1 = re.compile('[0-9]+[.]?[0-9]+')

def clean_text(text):
    text = re_perc_1.sub(' <percent> ', text)
    text = re_curr_2.sub(' <currency> ', text)
    text = re_curr_1.sub(' <currency> ', text)
    text = re_curr_4.sub(' <currency> ', text)
    text = re_curr_3.sub(' <currency> ', text)
    text = re_numb_1.sub(' <number> ', text)
    
    text = re.sub('[.]+\s*[.]+', '. ', text)
    text = re.sub('\n', ' ', text)
    text = re.sub('\s+', ' ', text)
    
    return text

In [4]:
if RESTART:
    with open(os.path.join(DATA_DIR, 'story_processed.txt'), 'w'):
        None
        
try:
    prev_story_log = open(os.path.join(DATA_DIR, 'story_processed.txt'), 'r').readlines()
    prev_story_log = [re.sub(r'\n', '', sl) for sl in prev_story_log]
except:
    prev_story_log = []

process cnn stories

In [5]:
cnn_stories = os.listdir(os.path.join(DATA_DIR, 'cnn'))

stories = []
highlights = []

with open(os.path.join(DATA_DIR, 'story_processed.txt'), 'a') as story_log:
    for story_fp in tqdm_notebook(cnn_stories):
        full_story_fp = os.path.join(DATA_DIR, 'cnn', story_fp)
        
        if full_story_fp not in prev_story_log:        
            story_log.write(full_story_fp+'\n')
            text = open(full_story_fp, 'r', encoding='utf8').readlines()    

            story = []
            highlight = []
            mode = 'story'

            # extract text
            for t in text:
                if '@highlight' in t:
                    mode = 'highlight'

                if mode == 'highlight' and '@highlight' not in t and t != '\n':
                    highlight.append(t)

                elif mode == 'story' and t != '\n':
                    story.append(t)

            # combine text
            story = '. '.join(story)    
            story = clean_text(story)

            highlight = '. '.join(highlight)
            highlight = clean_text(highlight)    

            # tokenize
            story = story.lower().split()
            highlight = highlight.lower().split()

            story = [''.join([w for w in re.findall('<[a-z]+>|[a-z]+', s) if len(w) != 0]) for s in story]
            highlight = [''.join([w for w in re.findall('<[a-z]+>|[a-z]+', s) if len(w) != 0]) for s in highlight]
            highlight = ['<sos>'] + highlight + ['<eos>']

            story = [s for s in story if s != '']
            highlight = [s for s in highlight if s != '']

            stories.append(story)
            highlights.append(highlight)

            if len(stories) % BATCH_SIZE == 0:
                filename = len(os.listdir(SAVE_PKL_DIR)) + 1
                filename = 'story_highlight_{}.pkl'.format(filename)        
                obj = [stories, highlights]

                pickle.dump(obj, open(os.path.join(SAVE_PKL_DIR, filename), 'wb'))

                stories = []
                highlights = []

if len(stories) != 0:
    filename = len(os.listdir(SAVE_PKL_DIR)) + 1
    filename = 'story_highlight_{}.pkl'.format(filename)        
    obj = [stories, highlights]

    pickle.dump(obj, open(os.path.join(SAVE_PKL_DIR, filename), 'wb'))

    stories = []
    highlights = []

process dailymail stories

In [6]:
dailymail_stories = os.listdir(os.path.join(DATA_DIR, 'dailymail'))

stories = []
highlights = []

with open(os.path.join(DATA_DIR, 'story_processed.txt'), 'a') as story_log:
    for story_fp in tqdm_notebook(dailymail_stories):        
        full_story_fp = os.path.join(DATA_DIR, 'dailymail', story_fp)
        
        if full_story_fp not in prev_story_log:        
            story_log.write(full_story_fp+'\n')
            text = open(full_story_fp, 'r', encoding='utf8').readlines()

            story = []
            highlight = []
            mode = 'story'

            # extract text
            for t in text:
                if '@highlight' in t:
                    mode = 'highlight'

                if mode == 'highlight' and '@highlight' not in t and t != '\n':
                    highlight.append(t)

                elif mode == 'story' and t != '\n':
                    story.append(t)

            # combine text
            story = '. '.join(story)    
            story = clean_text(story)

            highlight = '. '.join(highlight)
            highlight = clean_text(highlight)    

            # tokenize
            story = story.lower().split()
            highlight = highlight.lower().split()

            story = [''.join([w for w in re.findall('<[a-z]+>|[a-z]+', s) if len(w) != 0]) for s in story]
            highlight = [''.join([w for w in re.findall('<[a-z]+>|[a-z]+', s) if len(w) != 0]) for s in highlight]
            highlight = ['<sos>'] + highlight + ['<eos>']

            story = [s for s in story if s != '']
            highlight = [s for s in highlight if s != '']

            stories.append(story)
            highlights.append(highlight)

            if len(stories) % BATCH_SIZE == 0:
                filename = len(os.listdir(SAVE_PKL_DIR)) + 1
                filename = 'story_highlight_{}.pkl'.format(filename)        
                obj = [stories, highlights]

                pickle.dump(obj, open(os.path.join(SAVE_PKL_DIR, filename), 'wb'))

                stories = []
                highlights = []

if len(stories) != 0:
    filename = len(os.listdir(SAVE_PKL_DIR)) + 1
    filename = 'story_highlight_{}.pkl'.format(filename)        
    obj = [stories, highlights]

    pickle.dump(obj, open(os.path.join(SAVE_PKL_DIR, filename), 'wb'))

    stories = []
    highlights = []